In [3]:
#Fall 2017 Applied Data Science | Indiana University | Laura Kahn | Assignment 9

#pip install TwitterAPI
#pip install nltk, pip install stop-words, pip install pyquery

import json
import pandas as pd
import numpy as np
import sklearn
from sklearn.feature_extraction import text as sk_fe_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
from TwitterAPI import TwitterAPI

#Set up the variables for your 'application'
consumerkey = 'rPA7yZAP5902lpj1K4Jd2342N'
consumersecret = 'HhKcMrY0TxLTtkNsie2bmMvtKZTl7wYAMVe56E1EI5FsQMTVeo'

#Setup your API key
api = TwitterAPI(consumerkey,consumersecret,auth_type='oAuth2')

In [4]:
#Get metrics for models
def score(true,pred):
  return (accuracy_score(true,pred),
          precision_score(true,pred),
          recall_score(true,pred),
          f1_score(true,pred))

def print_score(s):
  print("""
Accuracy:   {:0.3}
Precision:  {:0.3}
Recall:     {:0.3}
F-Score:    {:0.3}
""".format(*s))

In [5]:
#Twitter Search Criteria
def searchTwitter(query,feed="search/tweets",api=api,n=1000):
  return [t for t in api.request(feed, {'q':query,'count':n})]

In [6]:
#Get JSON from Twitter search
cats = searchTwitter('#cats')
dogs = searchTwitter('#dogs')

In [16]:
# Convert the json returned by Twitter into a dataframe
cats_d = pd.read_json(json.dumps(cats))
dogs_d = pd.read_json(json.dumps(dogs))

In [29]:
# If you would like to look at the full data frame
# cats_d.info()
# dogs_d.info()

In [8]:
#Text Preprocessing - Get text only and replace hashtags with blanks
#If you want to use the normalizer, import it above and pass x.replace() to the noramlizer function

cats_text = [x.replace('#cats',"") for x in cats_d['text']]
dogs_text = [x.replace('#dogs',"") for x in dogs_d['text']]

In [9]:
#Let's take a look at the Tweets that contain the word "cat"
cats_text

['RT @catsskitten: Yoga Cat ❤️❤️\nType "Yes" If you love cats👇👇\nTag a friend who💟💟cats\n#Cats #cat ofinstagram #kitty #kitten #kittycat #C…',
 'Relaxed mood #animal #picoftheday #photography #animals #animalsofinstagram #awesome_shots #cat  #cute #igs… https://t.co/mdNyUS9cBC',
 'RT @catsskitten: Yoga Cat ❤️❤️\nType "Yes" If you love cats👇👇\nTag a friend who💟💟cats\n#Cats #cat ofinstagram #kitty #kitten #kittycat #C…',
 'RT @catsskitten: Type "Yes" If you love cats👇👇\nTag a friend who💟💟cats\n#Cats #cat ofinstagram #kitty #kitten #kittycat #CatsOfTwitter h…',
 'RT @RileyChi2: .@RepShimkus❗️#DogsSupportUs PLS 🙏🏽#SupportDogs and #Cats❗️ #Cosponsor #HR1406 The #DogCatMeatTradeProhibitionActOf2017  #Pr…',
 'Simon is too comfortable. I am actually trying to get ready for work.  #catparents https://t.co/7ajbTVYJ2s',
 'Jag har använt/I’ve used Too faced sha  #beautiful https://t.co/C76WP2Xu1g https://t.co/Wu8lLO4pwM',
 'RT @ngremmy: #URGENT #NYC #CATS precious long-haired tuxie PUFFIN abandone

In [30]:
#Create features and return sparse matricies
vectorizer = sk_fe_text.CountVectorizer(cats_text+dogs_text)
vectorizer.fit(cats_text+dogs_text)
cats_tdm = vectorizer.transform(cats_text).toarray()
dogs_tdm = vectorizer.transform(dogs_text).toarray()

In [31]:
#Create visible matricies and combine
zeros = np.zeros((len(cats_text),1))
ones = np.ones((len(dogs_text),1))
catsdogs = np.concatenate((cats_tdm,dogs_tdm),axis=0)
y = np.ravel(np.concatenate((zeros,ones),axis=0))

In [32]:
#Create train/test split for modeling
trainX,testX,trainY,testY = train_test_split(catsdogs,y,test_size=.20)

In [35]:
#Begin Classifier Modeling, Fitting, Performance Metrics

#Naive Bayes
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(trainX,trainY)

print("\n\nNaive Bayes Performance")
s = score(testY,nb.predict(testX))
print_score(s)



Naive Bayes Performance

Accuracy:   0.811
Precision:  0.765
Recall:     0.812
F-Score:    0.788



In [44]:
#Support Vector Machine, SVM

from sklearn import svm
svm = svm.SVC()
svm.fit(trainX,trainY)

print("\n\nSVM Performance")
s = score(testY,y_pred)
print_score(s)

#Not quite sure why the precision, recall and f-scores aren't working but they aren't relevant to the assignment
#question.



SVM Performance

Accuracy:   0.568
Precision:  0.0
Recall:     0.0
F-Score:    0.0



C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [40]:
#Neural Network Classifier
from sklearn.neural_network import MLPClassifier

nn = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5,2), random_state=1)
nn.fit(trainX,trainY)

print("\n\nNaive Bayes Performance")
s = score(testY,nn.predict(testX))
print_score(s)



Naive Bayes Performance

Accuracy:   0.919
Precision:  0.882
Recall:     0.938
F-Score:    0.909



#The accuracy is a lot higher for the Neural Network Classifier (0.919). The Naive Bayes classifier is the next
#most accurate model at 0.811, followed by the Support Vector Machine which is only slightly better than a randomly
#chosen baseline at 0.568.